# Object Extraction
Create a template SPARQL query for finding the lifted pre-conditions and effects of a user (NL) query.
   1. Extract the object from the user query (grounded version). Ex. Extract “unstack b2 b1” as the object.
   2. Get the pre-conditions and effects of the lifted version of the object. Ex. Finding the details of “unstack”.

In [1]:
import rdflib
from code.templates import rdf_utils
from typing import List

## Extract Grounded Objects from User Queries 

Given a user natural language (NL) query, extract the grounded object(s) from the user query. Some examples: 

- extract “moveLeft sokoban x1 y1” from the NL query “Why is moveLeft sokoban x1 y1 not used in the plan?”.
- extract “moveLeft sokoban x1 y1” from the NL query “Why is moveLeft sokoban x1 y1 used in the plan?”.
- extract both “moveLeft sokoban x1 y1” and “moveRight sokoban x1 y1” from the NL query “Why is moveLeft sokoban x1 y1 used rather than moveRight sokoban x1 y1?”.

Possible grounded objects for Sokoban: moveright, moveleft, moveup, movedown, pushright, pushleft, pushup, pushdown

In [2]:
file_path: str = "../../data/sokoban/plan-ontology-rdf-instances_sokoban.owl"
g: rdflib.graph.Graph = rdflib.Graph().parse(file_path, format="xml")

In [3]:
# Get all actions
actions: List[str] = rdf_utils.get_actions_from_rdf(g, 'sokoban')
actions

['moveleft',
 'moveright',
 'moveup',
 'movedown',
 'pushleft',
 'pushright',
 'pushup',
 'pushdown']

In [4]:
# Create a dictionary for the actions and their number of parameters 
# Keys: actions (string); Values: number of parameters (int)
actions_dict: dict[str, int] = {action: 0 for action in actions}

In [5]:
# Get parameters for the actions
for action in actions_dict.keys():
    parameters: List[str] = rdf_utils.get_parameters_from_rdf(g, action)
    actions_dict[action] = len(parameters)  # Update the number of parameters

actions_dict

{'moveleft': 3,
 'moveright': 3,
 'moveup': 3,
 'movedown': 3,
 'pushleft': 5,
 'pushright': 5,
 'pushup': 5,
 'pushdown': 5}

In [6]:
# Test examples
queries = [
    "Why is moveLeft sokoban x1 y1 not used in the plan?",
    "Why is moveLeft sokoban x1 y1 used rather than moveRight sokoban x1 y1?",
    "Why is moveup sokoban x1 y1 z1 used rather than pushdown sokoban x1 y1 a1 b1?",
]

# Extract grounded objects for each query
extracted_objects = [rdf_utils.extract_grounded_objects(query, actions_dict) for query in queries]

# Print results
for i, query in enumerate(queries):
    print(f"Query: '{query}'\nExtracted: {extracted_objects[i]}\n")

Query: 'Why is moveLeft sokoban x1 y1 not used in the plan?'
Extracted: ['moveLeft sokoban x1 y1']

Query: 'Why is moveLeft sokoban x1 y1 used rather than moveRight sokoban x1 y1?'
Extracted: ['moveLeft sokoban x1 y1', 'moveRight sokoban x1 y1']

Query: 'Why is moveup sokoban x1 y1 z1 used rather than pushdown sokoban x1 y1 a1 b1?'
Extracted: ['moveup sokoban x1 y1', 'pushdown sokoban x1 y1 a1 b1']



## Get Grounded Predicates
Given the grounded object(s) extracted from the user query, get the grounded pre-conditions and effects of the object(s).

In [7]:
for objects in extracted_objects:
    for obj in objects:  # Since there can be multiple objects
        preconditions, effects = rdf_utils.get_grounded_predicates(obj, g)
        print(f"Object: {obj}\nPreconditions: {preconditions}\nEffects: {effects}\n")

Object: moveLeft sokoban x1 y1
Preconditions: ['sokoban sokoban', 'at sokoban x1', 'leftOf y1 x1', 'clear y1']
Effects: ['at sokoban y1', 'clear x1', 'not (at sokoban x1)', 'not (clear y1)']

Object: moveLeft sokoban x1 y1
Preconditions: ['sokoban sokoban', 'at sokoban x1', 'leftOf y1 x1', 'clear y1']
Effects: ['at sokoban y1', 'clear x1', 'not (at sokoban x1)', 'not (clear y1)']

Object: moveRight sokoban x1 y1
Preconditions: ['sokoban sokoban', 'at sokoban x1', 'leftOf x1 y1', 'clear y1']
Effects: ['at sokoban y1', 'clear x1', 'not (at sokoban x1)', 'not (clear y1)']

Object: moveup sokoban x1 y1
Preconditions: ['sokoban sokoban', 'at sokoban x1', 'below x1 y1', 'clear y1']
Effects: ['at sokoban y1', 'clear x1', 'not (at sokoban x1)', 'not (clear y1)']

Object: pushdown sokoban x1 y1 a1 b1
Preconditions: ['sokoban sokoban', 'crate b1', 'below y1 x1', 'below a1 y1', 'at sokoban x1', 'at b1 y1', 'clear a1']
Effects: ['at sokoban y1', 'at b1 a1', 'clear x1', 'not (at sokoban x1)', 'not 